In [1]:
import pyupbit
import time
import json
import requests
import pandas as pd
import mysql.connector
from datetime import datetime, timedelta

##### DB연결

In [2]:
# MySQL 서버에 연결
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="deepCoin"
)

# 연결이 제대로 되었는지 확인
if mydb.is_connected():
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
else:
    print("MySQL 데이터베이스 연결에 실패했습니다.")

# 커서 생성
mycursor = mydb.cursor()

MySQL 데이터베이스에 성공적으로 연결되었습니다.


##### 종목마스터 수집

In [3]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
# columns = pyupbit.get_tickers(fiat="KRW")

url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)


# DB생성용 Master 항목당 최대 길이 출력
max_length = df['market'].str.len().max()
max_string = df[df['market'].str.len() == max_length]['market'].iloc[0]
# print(max_string, len(max_string))

# 마스터에 저장된 종목리스트
mst_market_list = df['market'].tolist()

# DataFrame을 튜플의 리스트로 변환
values = df.values.tolist()

# 컬럼명 리스트
columns = df.columns.tolist()

# 삽입할 테이블 이름
table_name = "c_krw_mst"

# 새로 상장된 종목 유무 확인
new_market = [x for x in mst_market_list if x not in market_list]

if(len(new_market) > 0):
    # 삽입할 테이블 이름
    table_name = "c_krw_mst"

    # 데이터 삽입 쿼리 생성
    sql = "INSERT INTO {} ({}) VALUES ({})".format(
        table_name,
        ', '.join(columns),
        ', '.join(['%s'] * len(columns))
    )

    # 데이터 삽입
    mycursor.executemany(sql, new_market)

    # 변경 사항 커밋
    mydb.commit()
    print(f"신규 종목 : {new_market}")
    print(mycursor.rowcount, "레코드가 삽입되었습니다.")
    mydb.close()
else:
    print("신규 상장 종목 없음")
    pass


신규 상장 종목 없음


##### [전일데이터] 전일 일봉데이터 가져오기

In [17]:
# 현재 날짜와 시간을 구합니다.
today = datetime.now()

# timedelta를 사용하여 어제의 날짜를 구합니다.
yesterday = today - timedelta(days=1)
dayBeforeYesterday = today - timedelta(days=2)

# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
columns = ['open', 'high', 'low', 'close', 'volume', 'value']

# 현재 날짜와 시간을 구합니다.
today = datetime.now()

# timedelta를 사용하여 어제의 날짜를 구합니다.
yesterday = today - timedelta(days=1)

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 어제 날짜 기준 일봉데이터 전체 호출
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=dayBeforeYesterday,
                   to=yesterday, period=0.5)
    date = yesterday.strftime('%Y-%m-%d')

    tuple_list = [tuple(x) for x in df.to_numpy()]

    try:
        # 데이터 삽입 쿼리 생성
        sql = "INSERT INTO {} ({}) VALUES ({})".format(
            table_name,
            ', '.join(columns),
            ', '.join(['%s'] * len(columns))
        )

        # 데이터 Insert
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.")
    except:
        # 데이터 업데이트 쿼리 생성
        sql = "UPDATE {} SET {} WHERE date = '{}' AND market = '{}'".format(
            table_name,
            ', '.join(['{} = %s'.format(column) for column in columns]),
            date, 
            market
            )
        # 데이터 Update
        mycursor.executemany(sql, tuple_list)
        print(mycursor.rowcount, f"{market} 레코드가 업데이트되었습니다.")

# 변경 사항 커밋
mydb.commit()
print("commit")

0 KRW-1INCH 레코드가 업데이트되었습니다.
0 KRW-AAVE 레코드가 업데이트되었습니다.
0 KRW-ADA 레코드가 업데이트되었습니다.
0 KRW-AERGO 레코드가 업데이트되었습니다.
0 KRW-AHT 레코드가 업데이트되었습니다.
0 KRW-ALGO 레코드가 업데이트되었습니다.
0 KRW-ANKR 레코드가 업데이트되었습니다.
0 KRW-APT 레코드가 업데이트되었습니다.
0 KRW-AQT 레코드가 업데이트되었습니다.
0 KRW-ARB 레코드가 업데이트되었습니다.
0 KRW-ARDR 레코드가 업데이트되었습니다.
0 KRW-ARK 레코드가 업데이트되었습니다.
0 KRW-ASTR 레코드가 업데이트되었습니다.
0 KRW-ATOM 레코드가 업데이트되었습니다.
0 KRW-AVAX 레코드가 업데이트되었습니다.
0 KRW-AXS 레코드가 업데이트되었습니다.
0 KRW-BAT 레코드가 업데이트되었습니다.
0 KRW-BCH 레코드가 업데이트되었습니다.
0 KRW-BLUR 레코드가 업데이트되었습니다.
0 KRW-BORA 레코드가 업데이트되었습니다.
0 KRW-BSV 레코드가 업데이트되었습니다.
0 KRW-BTC 레코드가 업데이트되었습니다.
0 KRW-BTG 레코드가 업데이트되었습니다.
0 KRW-BTT 레코드가 업데이트되었습니다.
0 KRW-CBK 레코드가 업데이트되었습니다.
0 KRW-CELO 레코드가 업데이트되었습니다.
0 KRW-CHZ 레코드가 업데이트되었습니다.
0 KRW-CRE 레코드가 업데이트되었습니다.
0 KRW-CRO 레코드가 업데이트되었습니다.
0 KRW-CTC 레코드가 업데이트되었습니다.
0 KRW-CVC 레코드가 업데이트되었습니다.
0 KRW-DKA 레코드가 업데이트되었습니다.
0 KRW-DOGE 레코드가 업데이트되었습니다.
0 KRW-DOT 레코드가 업데이트되었습니다.
0 KRW-EGLD 레코드가 업데이트되었습니다.
0 KRW-ELF 레코드가 업데이트되었습니다.
0 KRW-EOS 레코드가 업데이트되었습니다.
0 KRW-ETC 레코드가 업데이트되었습

##### [과거데이터] 최초 종목별 일봉데이터 가져오기

In [49]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 현재 날짜와 시간을 구합니다.
today = datetime.now()

# timedelta를 사용하여 어제의 날짜를 구합니다.
yesterday = today - timedelta(days=1)

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 어제 날짜 기준 일봉데이터 전체 호출
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=yesterday, period=0.5)
    df['date'] = df.index.strftime('%Y-%m-%d')
    df['market'] = market
    df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

    tuple_list = [tuple(x) for x in df.to_numpy()]

    # 데이터 삽입 쿼리 생성
    sql = "INSERT INTO {} ({}) VALUES ({})".format(
        table_name,
        ', '.join(columns),
        ', '.join(['%s'] * len(columns))
    )
    # 데이터 삽입
    mycursor.executemany(sql, tuple_list)
    print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.")

# 변경 사항 커밋
mydb.commit()
print("전체 레코드가 삽입되었습니다.")

IntegrityError: 1062 (23000): Duplicate entry '2021-10-27-KRW-1INCH' for key 'PRIMARY'